In [ ]:
# IMPORT PACKAGES, DATA, AND FUNCTIONS FROM SEPARATE FILE
from package_bfry import *

In [ ]:
# DESCRIPTIVE EXPLORATION #
# create maps that explore the geographic variation of the metrics produced 

In [ ]:
# HOUSEHOLD SIZE
# map the household size
base_map_2019.explore(column = 'avg_household_size', cmap = 'RdBu_r', tooltip = ('zona_fiu', 'population', 'households', 'avg_household_size'), 
                                          tiles = 'CartoDB positron', legend=True)

In [ ]:
# explore gyms per 1000
amenities_2019.explore(column = 'gyms_per_1000', cmap = 'RdBu', tooltip = ('zona_fiu', 'population', 'gyms_per_1000'), 
                                          tiles = 'CartoDB positron', legend=True)

In [ ]:
# airbnb #
# base dataset : airbnb_agg

create_map(base_map_2019, airbnb_agg, 'household', 'airbnb', airbnb)

In [ ]:
# WIFI #
# base dataset : wifi_agg

# map the wifi per capita
create_map(base_map_2019, wifi_agg, 'population', 'hotspot', wifi, quotient = 1000)

In [ ]:
# PARTICIPATORY BUDGET #
# base dataset : budg_geo

    # process data for mapping
# aggregate the count of participative budget projects at the neighborhood level
# TO DO - resolve mixed geometry situation
#budg_agg = 
#gpd.overlay(base_map_2019, budg_geo, how='intersection', keep_geom_type=False) #.groupby('cod_zona').count()
#budg_agg
# rename and subset the data just to the count of the projects per neighborhood
#budg_agg = budg_agg.rename(columns={'Progetto':'project_count'})['project_count']
budg_geo.explore()
    # map the data
#create_map(base_map_2019, budg_agg, 'population', 'project', budg_geo, quotient = 1000)

In [ ]:
# explore traffic per capita at the zone level
transport_2019.explore(column = 'traffic_per_1000', cmap = 'RdBu_r')

In [ ]:
# explore bike parking per capita / household
transport_2019.explore(column = 'bike_parking_per_household', cmap = 'RdBu_r')

In [ ]:
# explore bus stops per capita
transport_2019.explore(column = 'tper_stops_per_1000', cmap = 'RdBu_r')

In [ ]:
# calculate the number of traffic coils per zone to determine the adequacy of this measurement 
base_map_2019.join(traffic_2019_geo.sjoin(base_map_2019[['geometry']]).groupby('index_right')['day_total_traffic'].count()).explore(column = 'day_total_traffic', tooltip= ('day_total_traffic'), cmap = 'RdBu_r')

In [ ]:
# MULTI VARIABLE ANALYSIS #
    # explore and select metrics to use in categorization of Bologna's zones
    # create a dataframe of variables to be analyzed together
        # can add or remove metrics for analysis as needed
        # standardize the data by using z-scores - allows for simple comparison / analysis of above, at, or below average across the city's zones
    # run k-means cluster analysis on the standardized datasets
    # use principal components analysis (PCA) to complement the k-means clustering method - creating linear systems that explain as much variance as possible

In [ ]:
# calculate the z-score to give a more standard frame of rereference across metrics
df_z = (all_metrics - all_metrics.mean())/all_metrics.std()

# reorder columns for visualization purposes
cols = ['00-14', '15-29', '30-44', '45-64', '65 e oltre',
        'population', 'pop_density_km2', 'avg_income', 'workers_per_cap', 'students_per_cap',
        'traffic_per_1000', 'incident_per_traffic', 'injured_per_incident', 'mortality_per_incident', 
        'bike_parking_per_1000', 'bike_m_per_capita', 'percent_protected_bike', 'tper_stops_per_1000',
        'arredo_per_1000', 'p_arredo_good', 'gyms_per_1000', 'school_per_1000_child', 'wifi_per_1000', 'airbnb_per_household']
df_z = df_z[cols]

    # create a subset of just base/socioecon related metrics
df_z_socioecon = df_z[['population', '00-14', '15-29', '30-44', '45-64', '65 e oltre',
       'avg_income', 'workers_per_cap', 'students_per_cap', 'pop_density_km2']]

    # create a subset of just transport related metrics
df_z_transport = df_z[['injured_per_incident', 'mortality_per_incident',
       'incident_per_traffic', 'bike_parking_per_1000', 'bike_m_per_capita',
       'percent_protected_bike', 'tper_stops_per_1000']]

    # create a subset of just amenities related metrics
df_z_amenities = df_z[['p_arredo_good',
       'school_per_1000_child', 'arredo_per_1000', 'wifi_per_1000',
       'airbnb_per_household', 'gyms_per_1000']]

In [ ]:
# plot the distribution for each variable to explore variation
for column in df_z:
    df_z[[column]].plot.hist()

In [ ]:
# scattter matrix to view distributions and correlations together
pd.plotting.scatter_matrix(df_z_socioecon, diagonal="kde",figsize=(20,15))
plt.show()
#df_z
#hinton(df_z.corr())
df_z.corr()

In [ ]:
# visual test for best number of clusters
    # should use n-1 where n is the "elbow" of the plot where slope significantly changes
import matplotlib.pyplot as plt
%matplotlib inline

numClusters = [1,2,3,4,5,6,7,8]
SSE = []
for k in numClusters:
    k_means = cluster.KMeans(n_clusters=k)
    k_means.fit(df_z)
    SSE.append(k_means.inertia_)

plt.plot(numClusters, SSE)
plt.xlabel('Number of Clusters')
plt.ylabel('SSE')
plt.show()

In [ ]:
# create a 4-cluster model using all metrics
# this model will be the current subject of further research and visualization
df_z_clusters_4 = km_cluster_analysis(df_z, 4, base_map_2019)
# rename the clusters to have more meaningful titles
df_z_clusters_4.geo['Cluster'] = df_z_clusters_4.geo['Cluster'].str.replace('0', 'Eastern Outliers')
df_z_clusters_4.geo['Cluster'] = df_z_clusters_4.geo['Cluster'].str.replace('1', 'Outer Ring')
df_z_clusters_4.geo['Cluster'] = df_z_clusters_4.geo['Cluster'].str.replace('2', 'Inner Ring')
df_z_clusters_4.geo['Cluster'] = df_z_clusters_4.geo['Cluster'].str.replace('3', 'Center')
# rename the centroid clusters as well for the visualization-producing function
# create a new column in he centroids data table that holds the given titles
df_z_clusters_4.centroids['index'] = df_z_clusters_4.centroids.reset_index()['index'].replace(0, 'Eastern Outliers')
df_z_clusters_4.centroids['index'] = df_z_clusters_4.centroids.reset_index()['index'].replace(1, 'Outer Ring')
df_z_clusters_4.centroids['index'] = df_z_clusters_4.centroids.reset_index()['index'].replace(2, 'Inner Ring')
df_z_clusters_4.centroids['index'] = df_z_clusters_4.centroids.reset_index()['index'].replace(3, 'Center')
# set this new column as the table's index - this will help create the visualizations correctly
df_z_clusters_4.centroids = df_z_clusters_4.centroids.set_index('index')
# output the map
df_z_clusters_4.geo.explore(column = 'Cluster', cmap = ['red','blue','green','orange'], tiles= "CartoDB positron", tooltip = ('zona_fiu','Cluster'))

In [ ]:
# display the average z-score for each metric across each cluster (the centroids of each cluster)
# this tells us about the character of each cluster
# for interpretation: a more negative/positive number means a cluster is characterized by being more below/above the average across all zones
df_z_clusters_4.centroids

In [ ]:
for id in ['x','Eastern Outliers','Outer Ring','Inner Ring','Center']:
    for metrics_list in [['00-14', '15-29', '30-44', '45-64', '65 e oltre'],
                        ['population', 'pop_density_km2', 'avg_income', 'workers_per_cap', 'students_per_cap'],
                        ['traffic_per_1000', 'incident_per_traffic', 'injured_per_incident', 'mortality_per_incident'], 
                        ['bike_parking_per_1000', 'bike_m_per_capita', 'percent_protected_bike', 'tper_stops_per_1000'],
                        ['arredo_per_1000', 'p_arredo_good', 'gyms_per_1000', 'school_per_1000_child', 'wifi_per_1000', 'airbnb_per_household']]:
            # display the line charts
            cluster_line_chart(cluster_data = df_z_clusters_4, 
                analysis_data = df_z,
                cluster_id = id, 
                metrics = metrics_list)

In [ ]:
# calculate the average distance between the cluster members and the centroid
# this measure is an indicator of the strength of the relationship in the clusters
for id in ['Eastern Outliers','Outer Ring','Inner Ring','Center']:
    df_iter = df_z.join(df_z_clusters_4.geo['Cluster']).loc[df_z_clusters_4.geo['Cluster'] == id].drop(columns=['Cluster'])
    centroid = df_z_clusters_4.centroids.loc[df_z_clusters_4.centroids.index == id].to_numpy()[0]
    sum = 0
    for row in range(len(df_iter)):
        sum += dist(df_iter.iloc[row].to_numpy(),centroid)
    print(id + ' avg distance from centroid: ' + str(round(sum/len(df_iter),3)))

In [ ]:
# display the distance of each zone from its cluster's centroid
# this indicates how far away the zone is from the 'typical' zone of the cluster and will more easily show outliers
for id in ['Eastern Outliers','Outer Ring','Inner Ring','Center']:
    df_iter = df_z.join(df_z_clusters_4.geo['Cluster']).loc[df_z_clusters_4.geo['Cluster'] == id].drop(columns=['Cluster'])
    centroid = df_z_clusters_4.centroids.loc[df_z_clusters_4.centroids.index == id].to_numpy()[0]
    data = []
    for row in range(len(df_iter)):
        data.append([df_iter.index[row], dist(df_iter.iloc[row].to_numpy(),centroid)])
        #print(df_iter.index[row] + ' distance from centroid: ' + str(dist(df_iter.iloc[row].to_numpy(),centroid)))
    data.sort(key=lambda i: i[1], reverse = True)
    pprint(data)
    print('\n')

In [ ]:
# output charts focusing on croce del biacco - roveri as an outlier in the outer ring cluster
for metrics_list in [['00-14', '15-29', '30-44', '45-64', '65 e oltre'],
                        ['population', 'pop_density_km2', 'avg_income', 'workers_per_cap', 'students_per_cap'],
                        ['traffic_per_1000', 'incident_per_traffic', 'injured_per_incident', 'mortality_per_incident'], 
                        ['bike_parking_per_1000', 'bike_m_per_capita', 'percent_protected_bike', 'tper_stops_per_1000'],
                        ['arredo_per_1000', 'p_arredo_good', 'gyms_per_1000', 'school_per_1000_child', 'wifi_per_1000', 'airbnb_per_household']]:
            # display the line charts
            cluster_line_chart(cluster_data = df_z_clusters_4, 
                analysis_data = df_z.loc[df_z.index == 'CROCE DEL BIACCO - ROVERI'],
                cluster_id = 'Outer Ring', 
                metrics = metrics_list,
                colors = 'rainbow')

In [ ]:
# output charts focusing on croce del biacco - roveri as an outlier in the outer ring cluster
for metrics_list in [['00-14', '15-29', '30-44', '45-64', '65 e oltre'],
                        ['population', 'pop_density_km2', 'avg_income', 'workers_per_cap', 'students_per_cap'],
                        ['traffic_per_1000', 'incident_per_traffic', 'injured_per_incident', 'mortality_per_incident'], 
                        ['bike_parking_per_1000', 'bike_m_per_capita', 'percent_protected_bike', 'tper_stops_per_1000'],
                        ['arredo_per_1000', 'p_arredo_good', 'gyms_per_1000', 'school_per_1000_child', 'wifi_per_1000', 'airbnb_per_household']]:
            # display the line charts
            cluster_line_chart(cluster_data = df_z_clusters_4, 
                analysis_data = df_z.loc[df_z.index == 'OSSERVANZA - PADERNO'],
                cluster_id = 'Outer Ring', 
                metrics = metrics_list,
                colors = 'rainbow')

In [ ]:
# output charts focusing on croce del biacco - roveri as an outlier in the outer ring cluster
for metrics_list in [['00-14', '15-29', '30-44', '45-64', '65 e oltre'],
                        ['population', 'pop_density_km2', 'avg_income', 'workers_per_cap', 'students_per_cap'],
                        ['traffic_per_1000', 'incident_per_traffic', 'injured_per_incident', 'mortality_per_incident'], 
                        ['bike_parking_per_1000', 'bike_m_per_capita', 'percent_protected_bike', 'tper_stops_per_1000'],
                        ['arredo_per_1000', 'p_arredo_good', 'gyms_per_1000', 'school_per_1000_child', 'wifi_per_1000', 'airbnb_per_household']]:
            # display the line charts
            cluster_line_chart(cluster_data = df_z_clusters_4, 
                analysis_data = df_z.loc[df_z.index == 'SARAGOZZA - SAN LUCA'],
                cluster_id = 'Inner Ring', 
                metrics = metrics_list,
                colors = 'rainbow')

In [ ]:
# perform 4-cluster analysis on df_z_socioecon (socioecon metrics only) and display results
df_socio_clusters_4 = km_cluster_analysis(df_z_socioecon, 4, base_map_2019)
df_socio_clusters_4.geo.explore(column = 'Cluster', cmap = ['blue','orange','green','red'], tiles= "CartoDB positron", tooltip = ('zona_fiu','Cluster'))

In [ ]:
# display the centroids for the 4-cluster analysis of just socioeconomic metrics
df_socio_clusters_4.centroids

In [ ]:
for id in ['x','0','1','2','3']:
    for metrics_list in [['00-14', '15-29', '30-44', '45-64', '65 e oltre'],
                        ['population', 'pop_density_km2', 'inv_avg_household_size', 'avg_income', 'workers_per_cap', 'students_per_cap']]:
            # display the line charts
            cluster_line_chart(cluster_data = df_socio_clusters_4, 
                analysis_data = df_z_socioecon, 
                cluster_id = id, 
                metrics = metrics_list)

In [ ]:
# perform cluster analysis on df_z_transport (transport metrics only) and display results
# screeplot (the "elbow" graph) analysis of this dataset yielded a result that suggested 2 clusters was most meaningful 
df_transport_clusters_4 = km_cluster_analysis(df_z_transport, 4, base_map_2019)
df_transport_clusters_4.geo.explore(column = 'Cluster', cmap = ['blue','orange','green','red'], tiles= "CartoDB positron", tooltip = ('zona_fiu','Cluster'))
# should try to add more transportation related metrics and determine if this creates more meaningful insight

In [ ]:
# display the centroids for the 4-cluster analysis of just socioeconomic metrics
df_transport_clusters_4.centroids

In [ ]:
for id in ['x','0','1','2','3']:
    for metrics_list in [['traffic_per_1000', 'incident_per_1000', 'incident_per_traffic', 'injured_per_1000', 'injured_per_incident', 'mortality_per_1000'], 
                        ['bike_parking_per_1000', 'bike_m_per_capita', 'percent_protected_bike', 'tper_stops_per_1000']]:
            # display the line charts
            cluster_line_chart(cluster_data = df_transport_clusters_4, 
                analysis_data = df_z_transport, 
                cluster_id = id, 
                metrics = metrics_list)

In [ ]:
# perform 4-cluster analysis on df_z_socioecon (socioecon metrics only) and display results
df_amenities_clusters_4 = km_cluster_analysis(df_z_amenities, 4, base_map_2019)
df_amenities_clusters_4.geo.explore(column = 'Cluster', cmap = ['blue','orange','green','red'], tiles= "CartoDB positron", tooltip = ('zona_fiu','Cluster'))

In [ ]:
for id in ['x','0','1','2','3']:
    for metrics_list in [['p_arredo_good', 'arredo_per_1000', 'school_per_1000_child', 'airbnb_per_household', 'gyms_per_1000', 'wifi_per_1000']]:
            # display the line charts
            cluster_line_chart(cluster_data = df_amenities_clusters_4, 
                analysis_data = df_z_amenities, 
                cluster_id = id, 
                metrics = metrics_list)

In [ ]:
    # principal compoenents analysis
# set the resulting categories of the cluster analysis as dependent variable 
y = df_z_clusters_4.geo.reset_index()[['zona_fiu','Cluster']].sort_values(by = 'Cluster')

# process the dataset to allow cosnsistent labelling of clusters and sensable legend
# joins the Clusters to the analyzed dataframe, sorts by the Cluster given in the k-means cluster analysis, then removes the ID again for the PCA
df_z_pca = df_z.join(df_z_clusters_4.geo[['Cluster']]).reset_index().sort_values(by = 'Cluster').drop(columns=['Cluster']).set_index('zona_fiu')

# define the principal component analysis from the standardized z-score data
pca = PCA().fit(df_z_pca)

# display a scatterplot of the data to show the clusters in comparison to the first two principal components
pca_scatter(pca, df_z_pca, y)
# the PCA analysis and the cluster analysis appear compatible

In [ ]:
# run pca summary to see the amount of variance explained by each of the displayed PCs
pca_summary(pca, df_z)

In [ ]:
# display the coefficients associated with each metric in the principal components
# first principal component - round the coefficients to 3 decimal points
pca.components_[0].round(3)

In [ ]:
# repeat above for the second principal component
pca.components_[1].round(3)

In [ ]:
# display the columns to refernce against the coefficients in the last two cells
df_z.columns.to_array

In [ ]:
# hierarchical clustering - provides ability to see further granularity and inter-group similarities / differences compared to k-means

# average linkage - compares using average distance between members of each cluster
linkage_matrix = linkage(df_z, method='average', metric='euclidean')
plt.figure(figsize=(12, 6))
dendrogram(linkage_matrix, truncate_mode='lastp', p=20, leaf_rotation=90., leaf_font_size=8., show_contracted=True, labels=df_z.index)
plt.title('Hierarchical Clustering Dendrogram with Complete Linkage')
plt.xlabel('Data Points')
plt.ylabel('Distance')
plt.show()